In [49]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import numpy as np
# Module used to split string with different delimiters
import re

### Extract CSVs into DataFrames

In [50]:
# Reading csv files into python
Artist_File_Path = 'Resources/artistDf.csv'
Billboard_File_Path = 'Resources/billboardHot100_1999-2019_sample.csv'
Grammy_File_Path = 'Resources/grammySongs_1999-2019.csv'

# Making them into dataframe
Adf = pd.read_csv(Artist_File_Path)
Bdf = pd.read_csv(Billboard_File_Path)
Gdf = pd.read_csv(Grammy_File_Path)

In [51]:
def login():
    pw=input("Please enter your password for PGAdmin: ")
    return pw

### Transform DataFrames

In [52]:
# Eliminate extra columns:
#Artist Table
Adf=Adf[['Artist','Genres','NumAlbums','Gender']].copy() 

#Billboard Table
Bdf = Bdf[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart','Week','Genre']].copy()

#Grammy Table
Gdf=Gdf[['Artist','GrammyAward','GrammyYear','Name', 'Genre']].copy()

In [53]:
#changed all strings to lower case
Gdf = Gdf.astype(str).apply(lambda x: x.str.lower())
Adf = Adf.astype(str).apply(lambda x: x.str.lower())
Bdf = Bdf.astype(str).apply(lambda x: x.str.lower())

In [54]:
# Rename the column headers
Adf = Adf.rename(columns={'Artist':'artist', 
                          'Genres': 'genre',
                          'NumAlbums': 'num_albums',
                          'Gender': 'gender',
                          'Group.Solo': 'group_solo'})

# Clean the data by dropping duplicates and setting the index
Adf.drop_duplicates('artist', inplace=True)
Adf.set_index('artist', inplace=True)
Adf

,genre,num_albums,gender
artist,,,
ed sheeran,"pop,uk pop",8,m
justin bieber,"canadian pop,dance pop,pop,post-teen pop",10,m
jonas brothers,"boy band,dance pop,pop,post-teen pop",10,m
drake,"canadian hip hop,canadian pop,hip hop,pop rap,...",11,m
chris brown,"dance pop,pop,pop rap,r&b,rap",6,m
...,...,...,...
dwight yoakam,"country,country rock,nashville sound,outlaw co...",25,m
jordan knight,nan,4,m
lila mccann,"contemporary country,country,country dawn",4,f


In [55]:
#Transform Week column into year
Bdf['Week'] = pd.to_datetime(Bdf['Week'], format = '%Y-%m-%d').map(lambda x: x.strftime('%Y'))

# Rename the column headers
Bdf = Bdf.rename(columns={'Artists':'artist',
                          'Name': 'name_of_song',
                          'Peak.position': 'peak_position',
                          'Weeks.on.chart': 'weeks_on_chart',
                          'Week': 'year',
                          'Genre': 'genre'})


#Split artists at (,) and stack them in separate rows   
Bdf = Bdf.drop('artist', axis=1).join(Bdf['artist'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('artist'))

#Keep only first row when there are artist/song duplicate
Bdf.drop_duplicates(['artist', 'name_of_song'], keep='first', inplace=True)
Bdf['artist'].replace('', np.nan, inplace=True)
Bdf = Bdf.dropna()
Bdf.reset_index(drop=True, inplace=True)
Bdf

,name_of_song,peak_position,weeks_on_chart,year,genre,artist
0,old town road,1.0,7.0,2019,"country,atlanta,alternative country,hip-hop,tr...","lil nas,"
1,senorita,nan,nan,2019,pop,shawn mendes
2,senorita,nan,nan,2019,pop,camila cabello
3,bad guy,2.0,13.0,2019,"hip-hop,dark pop,house,trap,memes,alternative ...",billie eilish
4,talk,3.0,20.0,2019,"synth-pop,pop",khalid
...,...,...,...,...,...,...
2520,kiss me kiss me,nan,nan,2014,"australia,punk rock,pop-rock,pop,rock",5 seconds of summer
2521,afire love,nan,nan,2014,"rock,uk,pop",ed sheeran
2522,demons,6.0,61.0,2014,"adult alternative,adult contemporary,alternati...",imagine dragons
2523,beat of the music,44.0,20.0,2014,country,brett eldredge


In [56]:
# Rename the column headers
Gdf = Gdf.rename(columns={'Artist':'artist',
                          'GrammyAward': 'grammy_award',
                          'GrammyYear': 'grammy_year',
                          'Name': 'name_of_song',
                          'Genre': 'genre'})

#Replacing certain words with an empty space
Gdf['artist'] = Gdf['artist'].str.replace('songwriters','')
Gdf['artist'] = Gdf['artist'].str.replace('songwriter','')
Gdf['artist'] = Gdf['artist'].str.replace('artists','')
Gdf['artist'] = Gdf['artist'].str.replace('artist','')
Gdf['artist'] = Gdf['artist'].str.replace('soloists','')
Gdf['artist'] = Gdf['artist'].str.replace('soloist','')
Gdf['artist'] = Gdf['artist'].str.replace('(','')
Gdf['artist'] = Gdf['artist'].str.replace(')','')
Gdf['artist'] = Gdf['artist'].str.replace(u'\xa0', u'')

#Split artists at (,) and stack them in separate rows   
Gdf = Gdf.drop('artist', axis=1).join(Gdf ['artist'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('artist'))
Gdf = Gdf.drop('artist', axis=1).join(Gdf ['artist'].str.split('& |; |, |and |with |featuring',expand=True).stack().reset_index(level=1, drop=True).rename('artist'))
Gdf['artist'].replace('', np.nan, inplace=True)
Gdf.reset_index(drop=True, inplace=True)
Gdf = Gdf.drop_duplicates(['grammy_award','name_of_song','artist'], keep='first')
Gdf

,grammy_award,grammy_year,name_of_song,genre,artist
0,record of the year,2018,this is america,general,childish gambino
1,song of the year,2018,this is america,general,childish gambino
2,best pop solo performance,2018,joanne (where do you think you're goin'?),pop,lady gaga
3,best pop duo/group performance,2018,shallow,pop,lady gaga
4,best pop duo/group performance,2018,shallow,pop,bradley cooper
...,...,...,...,...,...
1736,best rap performance by a duo or group,1999,you got me,rap,the roots
1737,best rap performance by a duo or group,1999,you got me,rap,erykah badu
1738,best female country vocal performance,1999,man! i feel like a woman!,country,shania twain
1739,best male country vocal performance,1999,choices,country,george jones


### Merging tables

In [57]:
# Andrew and Marta did some rework on merging Bdf and Gdf. We filtered all the unque songs which got at least 
#1 grammy and were on Billboard, dropped artists (they don't matter).
B_G = pd.merge(Bdf,Gdf,on='name_of_song', how='inner')
del B_G['artist_y']
del B_G['artist_x']
B_G = B_G.drop_duplicates(['name_of_song','year'], keep='first')
B_G

,name_of_song,peak_position,weeks_on_chart,year,genre_x,grammy_award,grammy_year,genre_y
0,shallow,1.0,39.0,2019,"pop country,folk,musicals,screen,pop-rock,soun...",best pop duo/group performance,2018,pop
8,heaven,nan,nan,2019,"electro,edm,electro-pop,dance,dance-pop,electr...",best male r&b vocal performance,2006,r&b
10,heaven,15.0,27.0,2018,country,best male r&b vocal performance,2006,r&b
12,you say,29.0,43.0,2019,"worship,christian pop,pop,christian",best contemporary christian music performance/...,2018,gospel/contemporary christian music
37,tequila,21.0,50.0,2019,country,best country duo/group performance,2018,country
...,...,...,...,...,...,...,...,...
171,work it,2.0,26.0,2015,"hip-hop,rap",best female rap solo performance,2003,rap
172,get ur freak on,7.0,25.0,2015,"dance-pop,dance,hip-hop,alternative,rap",best rap solo performance,2001,rap
173,rather be,10.0,31.0,2015,"electro-pop,deep house,eurodance,dance-pop,cla...",best dance recording,2014,dance/electronic music
175,i'm not gonna miss you,nan,nan,2014,"screen,country",best country song,2014,country


In [58]:
B_Gdf=pd.merge(Bdf,Gdf,on='name_of_song')
B_Gdf=B_Gdf[['name_of_song', 'peak_position', 'weeks_on_chart', 'year', 'genre_y', 'artist_x', 'grammy_award', 'grammy_year']]
B_Gdf=B_Gdf.drop_duplicates()
B_Gdf=B_Gdf.groupby(['name_of_song','artist_x', 'peak_position', 'year', 'weeks_on_chart'])['grammy_award'].apply(', '.join).reset_index()
B_Gdf.rename(columns={'artist_x':'artist','genre_y':'genre'})
B_Gdf.tail(30)

,name_of_song,artist_x,peak_position,year,weeks_on_chart,grammy_award
39,imagine,ariana grande,21.0,2019,8.0,best pop collaboration with vocals
40,let it go,idina menzel,5.0,2014,33.0,best song written for visual media
41,let it go,james bay,16.0,2016,34.0,best song written for visual media
42,loyalty.,kendrick lamar,14.0,2017,26.0,best rap/sung performance
43,my boo,ghost town dj's,27.0,2016,36.0,best r&b performance by a duo or group with vo...
44,my church,maren morris,50.0,2016,20.0,best country solo performance
45,no problem,chance the rapper,43.0,2016,26.0,best rap performance
46,rather be,clean bandit,10.0,2015,31.0,best dance recording
47,redbone,childish gambino,12.0,2017,44.0,best traditional r&b performance
48,say something,justin timberlake,9.0,2018,16.0,best pop duo/group performance


### Load into database

In [59]:
import psycopg2
pw=login()
connection = None
try:
    # In PostgreSQL, default username is 'postgres' and password is 'postgres'.
    # And also there is a default database exist named as 'postgres'.
    # Default host is 'localhost' or '127.0.0.1'
    # And default port is '54322'.
    connection= psycopg2.connect(
      database='postgres', user='postgres', password=pw, host='127.0.0.1', port= '5432'
    )
    print('Database connected.')

except:
    print('Database not connected.')
#Check is the database exists or not
if connection is not None:
    connection.autocommit = True

    cur = connection.cursor()
    cur.execute("SELECT datname FROM pg_database;")

    list_database = cur.fetchall()
    if ('artist_db',) in list_database:
        connection.close()
        connection= psycopg2.connect(
        database='artist_db', user='postgres', password=pw, host='127.0.0.1', port= '5432'
        )
        cur = connection.cursor()
        print(" Database artist_db already exist, deleting all tables within the database")
        cur.execute("DROP SCHEMA public CASCADE;")
        cur.execute("CREATE SCHEMA public;")
        connection.close()
    else:
        print(" Database artist_db does not exist, creating one right now.......")
        sql = '''CREATE database artist_db'''
        cur.execute(sql)
        print("Database created successfully........")
        connection.close()
    print('Done')


Database connected.
 Database artist_db does not exist, creating one right now.......
Database created successfully........
Done


In [60]:
#Create connection
rds_connection_string = f"postgres:{pw}@localhost:5432/artist_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [62]:
#Append Dataframs as tables to artist_db
Adf.to_sql(name='artists', con=engine, if_exists='append', index=True)
Bdf.to_sql(name='billboard', con=engine, if_exists='append', index=False)
Gdf.to_sql(name='grammy_songs', con=engine, if_exists='append', index=False)
B_Gdf.to_sql(name='grammy_songs_joined_billboard', con=engine, if_exists='append', index=False)



In [66]:
#rank by genre, number of times it appears in data
query_1=engine.execute("select b.genre, count(b.genre) from billboard b group by b.genre order by count(b.genre) DESC").fetchall()
query_2=engine.execute("select g.genre, count(g.genre) from grammy_songs g group by g.genre order by count(g.genre) DESC").fetchall()

[('country', 562),
 ('rap', 372),
 ('trap,rap', 244),
 ('pop', 208),
 ('atlanta,trap,rap', 90),
 ('motown,trap,rap', 68),
 ('r&;b', 66),
 ('r&;b,rap', 64),
 ('r&;b,pop', 58),
 ('trap,canada,rap', 52),
 ('country,rock', 48),
 ('canada,rap', 46),
 ('uk,pop', 44),
 ('pop,rap', 36),
 ('cloud rap,trap,rap', 36),
 ('pop,r&;b', 34),
 ('east coast,rap', 30),
 ('rap,pop', 24),
 ('west coast,rap', 24),
 ('boy band,uk,pop', 24),
 ('canada,r&;b,electro-pop,pop', 22),
 ('atlanta,motown,trap,rap', 22),
 ('canada,pop', 18),
 ('hip-hop,rap', 18),
 ('r&;b,canada,rap', 18),
 ('dmv,trap,rap', 16),
 ('canada,r&;b', 14),
 ('pop,country', 14),
 ('country,pop', 14),
 ('trap,motown,rap', 14),
 ('r&;b,trap,rap', 14),
 ('rock,country', 14),
 ('electronic,dance,pop', 12),
 ('trap,east coast,rap', 12),
 ('rock', 12),
 ('emo rap,trap,rap', 12),
 ('canada,pop,r&;b', 12),
 ('disney,soundtrack,pop', 12),
 ('hip-hop,trap,rap', 10),
 ('christmas,pop', 10),
 ('canada,r&;b,rap', 10),
 ('electronic,pop', 10),
 ('pop,jamai

In [71]:
#rank by artist name, number of times they appear in data
query_3=engine.execute("select b.artist, count(b.artist) from billboard b group by b.artist order by count(b.artist) DESC").fetchall()
query_4=engine.execute("select g.artist, count(g.artist) from grammy_songs g group by g.artist order by count(g.artist) DESC").fetchall()

In [72]:
#join Artists and Billboard data
query_5=engine.execute("select b.artist, b.name_of_song, b.year , b.weeks_on_chart,   b.peak_position, a.num_albums, a.genre from billboard b inner join artists a on  a.artist = b.artist").fetchall()
#all join(raw)
query_6=engine.execute("select b.artist, b.year, b.weeks_on_chart,   a.genre, a.num_albums, g.grammy_award, g.grammy_year from billboard b inner join artists a on  a.artist = b.artist left join grammy_songs g on b.artist = g.artist").fetchall()

In [73]:
#all join
query_7=engine.execute("select g.artist, g.name_of_song, g.grammy_award, g.grammy_year, b.peak_position, a.genre from grammy_songs g left join billboard b on g.name_of_song = b.name_of_song inner join artists a on g.artist = a.artist").fetchall()
